# Advanced fMRI data analysis 
## General Linear Model 2


In this assignment, your task is to run simple GLM analysis on fMRI data in Python.

You can choose to answer in Polish or English (it will not affect your grade).

**DEADLINE:** 22-05-2020

-------------------

## Task 1



Repeat all analyses performed on our last classes (code [HERE](https://github.com/fMRIAnalysisCourse/fmri-analysis-course/blob/master/04-general_linear_model/glm_on_fMRI_data.ipynb)) on the second subject in our dataset (`sub-02`). Explore different plotting possibilities available at `plot_stat_map` method (documentation [HERE](https://nilearn.github.io/modules/generated/nilearn.plotting.plot_stat_map.html)). Also check tutorial [HERE](https://nilearn.github.io/auto_examples/01_plotting/plot_demo_more_plotting.html#sphx-glr-auto-examples-01-plotting-plot-demo-more-plotting-py).



In [ ]:
#Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from nistats.first_level_model import FirstLevelModel
from nistats.reporting import plot_design_matrix
from nistats.reporting import plot_contrast_matrix
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, show, view_img
from nistats.thresholding import map_threshold
from nistats.reporting import get_clusters_table


events_path = "../fmri-analysis-course-master/data/fMRI_BIDS_rhymejudgment/sub-02/func/sub-02_task-rhymejudgment_events.tsv"
events = pd.read_csv(events_path, sep="\t")
print(events.head())


#Loading fMRI image
fmri_img = "../fmri-analysis-course-master/data/fMRI_BIDS_rhymejudgment/derivatives/fmriprep/sub-02/func/sub-02_task-rhymejudgment_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
confounds_path = "../fmri-analysis-course-master/data/fMRI_BIDS_rhymejudgment/derivatives/fmriprep/sub-02/func/sub-02_task-rhymejudgment_desc-confounds_regressors.tsv"
confounds = pd.read_csv(confounds_path, sep="\t")
motion = confounds[["trans_x", "trans_y", "trans_z", "rot_x", "rot_y", "rot_z"]]


t_r = 2
first_level_model = FirstLevelModel(t_r, 
                                    hrf_model='spm', 
                                    high_pass=.01,
                                    smoothing_fwhm=6)

first_level_model = first_level_model.fit(fmri_img, events=events)


design_matrix = first_level_model.design_matrices_[0]

# Plot design matrix
plot_design_matrix(design_matrix)

In [ ]:
# Printing timeseries of data
plt.plot(design_matrix['pseudoword'])
plt.plot(design_matrix['word'])
plt.legend(design_matrix.columns[0:2])
plt.show()

In [ ]:
# Create conditions
conditions = {'pseudoword': np.array([1, 0, 0, 0, 0, 0, 0, 0, 0]), 
              'word': np.array([0, 1, 0, 0, 0, 0, 0, 0, 0])}

# Create contrasts
word_minus_pseudoword = conditions['word'] - conditions['pseudoword']
word_effect = conditions['word']

# Plot contrast matrix
plot_contrast_matrix(word_effect, design_matrix=design_matrix)

# Calculate statistic test for selected contrast
z_map = first_level_model.compute_contrast(word_effect,
                                  output_type='z_score')
plot_stat_map(z_map, threshold=5.179,
              display_mode='z', cut_coords=3, black_bg=True)
plt.show()

z_map1 = first_level_model.compute_contrast(word_minus_pseudoword,
                                  output_type='z_score')
plot_stat_map(z_map, threshold=3,
              display_mode='z', cut_coords=3, black_bg=True)
plt.show()

# Threshold z-matrices to correct for multiple comparisons
_, threshold = map_threshold(z_map, alpha=.05, height_control='bonferroni')
print('Uncorrected p<0.001 threshold: %.3f' % threshold)
plot_stat_map(z_map, threshold=threshold,
              display_mode='z', cut_coords=3, black_bg=True)
plt.show()

# Get cluster table
table = get_clusters_table(z_map, stat_threshold=threshold,
                           cluster_threshold=20)

# Repeat everything with motion as nuisance regressors.
first_level_model = first_level_model.fit(fmri_img, events=events, confounds=motion)

design_matrix1 = first_level_model.design_matrices_[0]
design_matrix1.head()

plot_design_matrix(design_matrix1)

conditions1 = conditions = {'pseudoword': np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 
              'word': np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'trans_y': np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

# Create contrasts
word_minus_pseudoword1 = conditions1['word'] - conditions1['pseudoword']
word_effect1 = conditions1['word']
trans_y = conditions1['trans_y']
# Plot contrast matrix
plot_contrast_matrix(word_minus_pseudoword1, design_matrix=design_matrix1)

z_map_mov = first_level_model.compute_contrast(word_effect1,
                                  output_type='z_score')
plot_stat_map(z_map_mov, threshold=3,
              display_mode='z', cut_coords=6, black_bg=True)
plt.show()


z_map_mov_trans = first_level_model.compute_contrast(trans_y,
                                  output_type='z_score')
plot_stat_map(z_map_mov_trans, threshold=3,
              display_mode='z', cut_coords=6, black_bg=True)
plt.show()

In [ ]:
#Sagittal map
plot_stat_map(z_map_mov_trans, threshold=3,
              display_mode='x', cut_coords=6, black_bg=True)
plt.show()

#coronal map
plot_stat_map(z_map_mov_trans, threshold=4,
              display_mode='y', cut_coords=6, black_bg=True)
plt.show()

#orthogonal
plot_stat_map(z_map_mov_trans, threshold=3,
              display_mode='ortho', cut_coords=(1, 1, 1), black_bg=True)
plt.show()

#stat map
plot_stat_map(z_map_mov_trans, threshold=3,
              display_mode='tiled', cut_coords=(1, 1, 50), black_bg=True)
plt.show()